In [1]:
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU, BatchNormalization

In [2]:
## loading the data
with open("1661-0.txt", "r") as file:
    text = file.read().lower()

In [3]:
## tokenizing the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

8923

In [4]:
## creating training sample where each sequence gets progressively increase
sequences = []
for line in text.split('.'):  ## spliting new sentences
    token_list = tokenizer.texts_to_sequences([line])[0]  ## converting word(sentence) to tokenized list
    for i in range (1,len(token_list)):  ## start from 1 bcs it ensure that list atleast have 2 words(token)
        n_gram_sequence = token_list[:i+1]    
        sequences.append(n_gram_sequence)


In [5]:
max_sequence_len = max(len(x) for x in sequences)
print(max_sequence_len)

## padding to sequence so that input size is fixed
input_sequence = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre')) 

105


In [6]:
x,y = input_sequence[:,:-1], input_sequence[:,-1]

In [7]:
y = tf.keras.utils.to_categorical(y, num_classes = total_words)

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42) 

In [9]:
from tensorflow.keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='accuracy',  restore_best_weights = True)

### using LSTM

In [9]:
## creating model using LSTM
model = Sequential([
    Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_len - 1),
    LSTM(100),
    BatchNormalization(),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dense(total_words, activation='softmax')  ## Predict next word
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 104, 100)          892300    
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 batch_normalization (BatchN  (None, 100)              400       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 8923)              901223    
                                                        

In [11]:
history = model.fit(x_train, y_train, epochs=120)

Epoch 1/120
2616/2616 [==============================] - 43s 14ms/step - loss: 6.3115 - accuracy: 0.0913
Epoch 2/120
2616/2616 [==============================] - 36s 14ms/step - loss: 5.5059 - accuracy: 0.1349
Epoch 3/120
2616/2616 [==============================] - 36s 14ms/step - loss: 5.1029 - accuracy: 0.1550
Epoch 4/120
2616/2616 [==============================] - 36s 14ms/step - loss: 4.7944 - accuracy: 0.1737
Epoch 5/120
2616/2616 [==============================] - 36s 14ms/step - loss: 4.5355 - accuracy: 0.1892
Epoch 6/120
2616/2616 [==============================] - 36s 14ms/step - loss: 4.3141 - accuracy: 0.2038
Epoch 7/120
2616/2616 [==============================] - 36s 14ms/step - loss: 4.1275 - accuracy: 0.2180
Epoch 8/120
2616/2616 [==============================] - 37s 14ms/step - loss: 3.9550 - accuracy: 0.2303
Epoch 9/120
2616/2616 [==============================] - 36s 14ms/step - loss: 3.8135 - accuracy: 0.2434
Epoch 10/120
2616/2616 [==============================]

In [10]:
def predict_next_word(model, input_text, tokenizer, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([input_text])[0]
    if len(token_list)>=max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted_max_index = np.argmax(predicted,axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_max_index:
            return word
        
    return None 

In [13]:
input_text = "The note was undated, and without either signature or"
word = predict_next_word(model=model, input_text=input_text, tokenizer=tokenizer, max_sequence_len=max_sequence_len)
print(f"{input_text} ,prediction: {word}")

1/1 [==============================] - 0s 472ms/step
The note was undated, and without either signature or ,prediction: address


In [14]:
input_text = "Peculiar—that is the very word,” said"
word = predict_next_word(model=model, input_text=input_text, tokenizer=tokenizer, max_sequence_len=max_sequence_len)
print(f"{input_text} ,prediction: {word}")

1/1 [==============================] - 0s 23ms/step
Peculiar—that is the very word,” said ,prediction: holmes


In [15]:
input_text = "“Very, indeed. And what does she propose to do with the"
word = predict_next_word(model=model, input_text=input_text, tokenizer=tokenizer, max_sequence_len=max_sequence_len)
print(f"{input_text} ,prediction: {word}")

1/1 [==============================] - 0s 17ms/step
“Very, indeed. And what does she propose to do with the ,prediction: photograph


In [16]:
import pickle
model.save("next_word_predition.h5")
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

### using Bidirectional with GRU

In [11]:
## creating model using bidirectional GRU
model_bidirectonal = Sequential([
    Embedding(input_dim=total_words, output_dim=50, input_length=max_sequence_len - 1),
    Bidirectional(GRU(128, return_sequences=True)),
    Dropout(0.2),
    BatchNormalization(),
    Bidirectional(GRU(64)),
    Dense(total_words, activation='softmax')  ## Predict next word
])

model_bidirectonal.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model_bidirectonal.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 104, 50)           446150    
                                                                 
 bidirectional (Bidirectiona  (None, 104, 256)         138240    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 104, 256)          0         
                                                                 
 batch_normalization (BatchN  (None, 104, 256)         1024      
 ormalization)                                                   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              123648    
 nal)                                                            
                                                        

In [13]:
history_bidirectional = model_bidirectonal.fit(x_train, y_train, epochs=120)

Epoch 1/120
2616/2616 [==============================] - 68s 23ms/step - loss: 6.4356 - accuracy: 0.0760
Epoch 2/120
2616/2616 [==============================] - 66s 25ms/step - loss: 5.6118 - accuracy: 0.1283
Epoch 3/120
2616/2616 [==============================] - 67s 26ms/step - loss: 5.2093 - accuracy: 0.1517
Epoch 4/120
2616/2616 [==============================] - 69s 26ms/step - loss: 4.8924 - accuracy: 0.1701
Epoch 5/120
2616/2616 [==============================] - 66s 25ms/step - loss: 4.6107 - accuracy: 0.1866
Epoch 6/120
2616/2616 [==============================] - 66s 25ms/step - loss: 4.3562 - accuracy: 0.2021
Epoch 7/120
2616/2616 [==============================] - 66s 25ms/step - loss: 4.1278 - accuracy: 0.2219
Epoch 8/120
2616/2616 [==============================] - 63s 24ms/step - loss: 3.9361 - accuracy: 0.2373
Epoch 9/120
2616/2616 [==============================] - 65s 25ms/step - loss: 3.7347 - accuracy: 0.2580
Epoch 10/120
2616/2616 [==============================]

In [14]:
input_text = "The note was undated, and without either signature or"
word = predict_next_word(model=model_bidirectonal, input_text=input_text, tokenizer=tokenizer, max_sequence_len=max_sequence_len)
print(f"{input_text} ,prediction: {word}")

1/1 [==============================] - 1s 758ms/step
The note was undated, and without either signature or ,prediction: address


In [15]:
input_text = "Peculiar—that is the very word,” said"
word = predict_next_word(model=model_bidirectonal, input_text=input_text, tokenizer=tokenizer, max_sequence_len=max_sequence_len)
print(f"{input_text} ,prediction: {word}")

1/1 [==============================] - 0s 22ms/step
Peculiar—that is the very word,” said ,prediction: he


In [16]:
input_text = "“Very, indeed. And what does she propose to do with the"
word = predict_next_word(model=model_bidirectonal, input_text=input_text, tokenizer=tokenizer, max_sequence_len=max_sequence_len)
print(f"{input_text} ,prediction: {word}")

1/1 [==============================] - 0s 22ms/step
“Very, indeed. And what does she propose to do with the ,prediction: photograph


In [17]:
import pickle
model_bidirectonal.save("next_word_predition_using_Bidirectional.h5")
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)